In [227]:
"""
Engineering-based cost model for PV manufacturing.
Based on Polysilicon Chunk Manufacturing Cost Excel Model

Manufacturing Steps: 
1. Harvest Chunk
2. Siemens CVD
3. Etch Filaments
4. Machine Filaments
5. Ingots (Saw, Crop, Anneal, Growth)
6. TCS

Cost summary adds cost from each step in manufacturing process

VERSION STATUS: Validated  all manufacturing steps and financial summary with Excel Model

FUTURE WORK: 
    - use named_tuples when reading in input variables to make it easier to create new models in the future
    - isolate administrative functions/code from calculations; could read in CSVs and return objects
    - convert True/False input file parameters to true boolean values (likely need to define function to do this)

"""


import numpy as np
import pandas as pd
import random
#import seaborn as sns
#import functions as fn
#import matplotlib.pyplot as plt

from copy      import deepcopy
from functools import reduce
from math      import exp, log

from Ipython.display import display

ModuleNotFoundError: No module named 'Ipython'

In [2]:
""" 
------ Define Functions for Manufacturing Step Costs -----
"""


def buildingCosts(floorSQF, clean10KSQF, clean1KSQF, clean100SQF):    
    return buildingCostSQF * floorSQF + CR10KCost * clean10KSQF + CR1KCost * clean1KSQF + CR100Cost * clean100SQF


def runtime(productionVol, throughput, downtime): 
    return (productionVol * 1000) / (throughput * hrs * days * (1 - downtime))


def parallelStationCalc(runtimeOneStation):
    return  np.ceil(runtimeOneStation) if dedicatedEquip == 'True' else runtimeOneStation.copy()


def toolLifeCalc(numParallelStations, toolingLife, effProductionVol): 
    toolLives = [plantLife, (numParallelStations * toolingLife / (effProductionVol * 1000))]
    return np.min(toolLives)


def grossIngotWeight(length, weightFraction, diameter):
    return np.pi * ((diameter / 20)**2 * length * 2.33 / 1000 * (1 + weightFraction))


def sizeFilamentBatch(diameter, filamentWidth, kerfLoss):
    size = np.trunc( (np.pi * (diameter / 20)**2) / ( (filamentWidth / 10) + (kerfLoss / 10000) )**2 )
    size = np.trunc(0.9 * size)
    return size


def dfSelect(df,idx,col):
    return df.loc[df.index == idx, col].item()


def skipStep(): 
    empty = pd.DataFrame({
            '$/kg Poly Si Chunk': [0,0,0,0,0,0,0,0,0], 
            '$/year' : [0,0,0,0,0,0,0,0,0]}, 
            index=['Material Cost','Direct Labor Cost','Utility Cost','Equipment Cost','Tooling Cost','Building Cost','Maintenance Cost','Overhead Labor Cost','Cost of Capital'])
    rejRate = 0
    return empty, rejRate


def financialSummary(dfs): 
    summary = pd.DataFrame()
    for df in dfs: 
        summary = summary.add(df, fill_value=0)
    summary['%'] = summary['$/year'] / summary['$/year'].sum()
    summary.loc['Total'] = summary.sum()
    return summary
    


def financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                          totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost):
    """
    Calculates the financial summary ($/kg-Poly Si and $/yr) of the manufacturing step. Outputs DataFrame of Values
    
    Returns numpy dataframe object with financial cost summary
    
    Parameters
    ----------
    capitalInvestment : float
        CapEx for the manufacturing process
    auxEquipInvest : float
        Auxilary Equipment for manufacturing process
    installFactor : float
        Installation cost mulitiplier
    runTimeOneStation : float
        Time one manufacturing station takes to run
    toolingInvestment : float
        Capital investment for tools in manufacturing step
    toolSetPerStation : float
        Number of tool sets required for each station
    buildingCostPerStation : float
        Cost of floor space for the station
    numParallelStations : float
        Number of stations required for parallel operation
    totalMaterialCosts : float
        Total cost of all input materials into the process
    unskilledDirectLaborers : float
        Number of unskilled laborers needed
    skilledDirectLaborers : float
        Number of skilled laborers needed
    throughput : float
        Rate of product creation for that manufacturing step
    avgDowntime : float
        Fraction of time the manufacturing equipment is spent not operating
    cumRejectionRate : float
        Cumulative fraction of material rejected (wasted) in the process
    operatingPowerConsumption : float
        Power requirement for the manufacutirng process
    equipMaintCost : float
        Equipment cost expressed as fraction of total investment
        
    """
    
    # calculate total costs to input into annuities and cost summary
    totalEquipInvest = capitalInvestment * (1 + auxEquipInvest) * (1 + installFactor) * np.ceil(runtimeOneStation)
    totalToolingInvest = toolinglInvestment * toolSetPerStation * np.ceil(runtimeOneStation)
    totalBuildingInvest = buildingCostperStation * np.ceil(runtimeOneStation)
    
    # calculate annuity payments
    equipAnnuity = np.pmt(capitalRecRate / 12, equipRecLife * 12, -totalEquipInvest) * (numParallelStations / np.ceil(runtimeOneStation)) * 12
    toolingAnnuity = np.pmt(capitalRecRate / 12, plantLife * 12, -totalToolingInvest) * 12
    buildingAnnuity = np.pmt(capitalRecRate / 12, buildingLife * 12, -totalBuildingInvest) * (numParallelStations / np.ceil(runtimeOneStation)) * 12
    
    # create dataframe with normalized costs and total costs
    costSummary = pd.DataFrame({
            '$/kg Poly Si Chunk': [
                    totalMaterialCosts,
                    (unskilledWage * (1 + benefitFactor) * unskilledDirectLaborers / throughput + skilledWage * (1 + benefitFactor) * skilledDirectLaborers / throughput) / ((1 - avgDowntime) * (1 - cumRejectionRate)),
                    operatingPowerConsumption * elecPrice / ((throughput * (1 - cumRejectionRate) * (1 - avgDowntime))), 
                    0,0,0,0,0,0], 
            '$/year' : [
                    0,0,0,
                    totalEquipInvest * numParallelStations / (equipRecLife * np.ceil(runtimeOneStation)), 
                    totalToolingInvest / plantLife, 
                    totalBuildingInvest * numParallelStations / (buildingLife * np.ceil(runtimeOneStation)), 
                    equipMaintCost * ((totalEquipInvest + totalToolingInvest) * numParallelStations / np.ceil(runtimeOneStation) + totalBuildingInvest), 
                    (unskilledDirectLaborers + skilledDirectLaborers) * numParallelStations * salary * indirectLabor * (1 + benefitFactor), 
                    0
                    ]}, 
            index=['Material Cost','Direct Labor Cost','Utility Cost','Equipment Cost','Tooling Cost','Building Cost','Maintenance Cost','Overhead Labor Cost','Cost of Capital'])
    
    variableCostRows = ['Material Cost','Direct Labor Cost','Utility Cost']        
    costSummary.loc[costSummary.index.isin(variableCostRows), '$/year'] = costSummary['$/kg Poly Si Chunk'] * annualProdVol * 1000
    
    workingRows = ['Material Cost','Direct Labor Cost','Utility Cost','Maintenance Cost','Overhead Labor Cost']
    workingAnnuity = np.pmt(capitalRecRate / 12, workingCapPeriod, -(workingCapPeriod * (costSummary.loc[costSummary.index.isin(workingRows), '$/year'].sum()) / 12)) * 12
    
    costSummary.loc[costSummary.index == 'Cost of Capital', '$/year'] = (equipAnnuity + toolingAnnuity + buildingAnnuity + workingAnnuity) - costSummary['$/year'].sum()
    
    fixedCostRows = ['Equipment Cost','Tooling Cost','Building Cost','Maintenance Cost','Overhead Labor Cost','Cost of Capital']
    costSummary.loc[costSummary.index.isin(fixedCostRows), '$/kg Poly Si Chunk'] = costSummary['$/year'] / (annualProdVol * 1000)
    
    return costSummary # dataframe object


def harvestChunk(inUse): 
    """
    Calculates cost of Harvest Chunk manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    """
    
    if inUse == 'False':
        costSummary, cumRejectionRate = skipStep()
    else: 
        
        # define input parameters
        
        argonGas                    = harvestChunkInputs['Value']['Argon Usage']                                        # SLM
        argonPrice                  = harvestChunkInputs['Value']['Argon Price']                                        # $/std liter
        argonScrapRate              = harvestChunkInputs['Value']['Argon Scrap Rate']                                   # %
        
        polySiYieldLossRate         = harvestChunkInputs['Value']['Poly Si Yield Loss']                                 # %
        polySiScrapReclRate         = harvestChunkInputs['Value']['Poly Si Scrap Reclemation Rate']                     # %
        avgDowntime                 = harvestChunkInputs['Value']['Average Downtime']                                   # %
        
        auxEquipInvest              = harvestChunkInputs['Value']['Auxiliary Equipment Investment']                     # % 
        installFactor               = harvestChunkInputs['Value']['Installation Cost Factor']
        equipMaintCost              = harvestChunkInputs['Value']['Equipment Maintenance Cost Factor']                  # %
        baseCapEx                   = harvestChunkInputs['Value']['Capital Investment']                                 # $/station
        
        unskilledDirectLaborers     = harvestChunkInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor     # ppl/station
        skilledDirectLaborers       = harvestChunkInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor       # ppl/station
        
        toolinglInvestment          = harvestChunkInputs['Value']['Tooling Investment']                                 # per toolset
        toolingLife                 = harvestChunkInputs['Value']['Tooling Life']                                       # kgs/tool
        
        operatingPowerConsumption   = harvestChunkInputs['Value']['Operating Power Consumption']                        # kW
        floorSQM                    = harvestChunkInputs['Value']['Floor SQM']                                          # sq m
        clean10KSQM                 = harvestChunkInputs['Value']['Clean 10k SQM']                                      # sq m
        clean1KSQM                  = harvestChunkInputs['Value']['Clean 1k SQM']                                       # sq m
        clean100SQM                 = harvestChunkInputs['Value']['Clean 100 SQM']                                      # sq m
        

        # intermediate calculations
        
        cumRejectionRate = 1 - (1 - polySiYieldLossRate * (1 - polySiScrapReclRate))
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        cycleTime = days * hrs * 60 / (effProductionVol * 1000)
        throughput = 60 / cycleTime
        
        materialCosts = argonGas * argonPrice * 60 / throughput
        totalMaterialCosts = materialCosts / ( (1 - cumRejectionRate) * (1 - argonScrapRate))       # material costs depend on regjection rate which is different for each mfg step
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = (1 - equipDiscount) * baseCapEx * annualProdVol * capexFactor / 1.7     # cost per station
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
        
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)
        
        
        # cost calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        return costSummary, cumRejectionRate


def siemensCVD(inUse, harvestChunkRejRate):
    """
    Calculates cost of Harvest Chunk manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    harvestChunkRejRate : float
        Fraction of material wasted in Harvest Chunk manufacturing step
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    reactorThroughput : float
        Througput of the reactor in the manufacturing step
    totalSiPerRod : float
        Total mass of Si per rod including tail/head
    usableSiPerRod : float
        Usable mass of Si per rod
    effProductionVol : float
        Effective production volume from the manufacturing step
    """
    
    if inUse == 'False':
        costSummary, cumRejectionRate = skipStep()
        reactorThroughput, totalSiPerRod, usableSiPerRod, effProductionVol = 0, 0, 0, 0
    else:
        
        # define siemens CVD mfg step input parameters

        h2Gas                       = siemensCVDInputs['Value']['Hydrogen Consumption Rate']                        # kg/hr-station
        h2Price                     = siemensCVDInputs['Value']['Hydrogen Price']                                   # $/kg
        h2ScrapRate                 = siemensCVDInputs['Value']['Hydrogen Scrap Rate']                              # %
        
        materialScrapRate           = siemensCVDInputs['Value']['Material Scrap Rate']                              # %
        polySiRejectRate            = siemensCVDInputs['Value']['Poly Si Reject Rate']                              # %
        polySiScrapReclRate         = siemensCVDInputs['Value']['Poly Si Scrap Rec Rate']                           # %
        avgDowntime                 = siemensCVDInputs['Value']['Average Downtime']                                 # %
        
        otherConsumableCost         = siemensCVDInputs['Value']['Other Consumable Cost']                            # $/kg PS
        batchSize                   = siemensCVDInputs['Value']['Batch Size']                                       # rods/batch
        finalRodSize                = siemensCVDInputs['Value']['Final Rod Size']                                   # mm diameter
        
        inputCycleTime              = siemensCVDInputs['Value']['Cycle Time']                                       # hrs/batch
        processTime                 = siemensCVDInputs['Value']['Setup, Harvest, Clean Time']                       # hrs/batch
        baseCapEx                   = siemensCVDInputs['Value']['Capital Investment']                               # $/station
        
        auxEquipInvest              = siemensCVDInputs['Value']['Auxiliary Equipment Investment']                   # % 
        installFactor               = siemensCVDInputs['Value']['Installation Cost Factor']
        equipMaintCost              = siemensCVDInputs['Value']['Equipment Maintenance Cost Factor']                # %
        
        unskilledDirectLaborers     = siemensCVDInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor   # ppl/station
        skilledDirectLaborers       = siemensCVDInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor     # ppl/station
        
        toolinglInvestment          = siemensCVDInputs['Value']['Tooling Investment']                               # per toolset
        toolingLife                 = siemensCVDInputs['Value']['Tooling Life']                                     # kgs/tool
        
        operatingPowerConsumption   = siemensCVDInputs['Value']['Operating Power Consumption']                      # kW
        floorSQM                    = siemensCVDInputs['Value']['Floor SQM']                                        # sq m
        clean10KSQM                 = siemensCVDInputs['Value']['Clean 10k SQM']                                    # sq m
        clean1KSQM                  = siemensCVDInputs['Value']['Clean 1k SQM']                                     # sq m
        clean100SQM                 = siemensCVDInputs['Value']['Clean 100 SQM']                                    # sq m
        
        
        # intermediate calculations
        
        cumRejectionRate = 1 - (1 - (materialScrapRate + polySiRejectRate) * (1 - polySiScrapReclRate))*(1 - harvestChunkRejRate *(1 - polySiScrapReclRate))
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        totalSiPerRod = (np.pi * (finalRodSize / 20)**2 * ingotGrowthInputs['Value']['Ingot Length'] * 2.33/1000) 
        usableSiPerRod = totalSiPerRod * (1 - materialScrapRate)
        
        cycleTime = (inputCycleTime + processTime) * 60 / (batchSize * usableSiPerRod)
        throughput = 60 / cycleTime
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        reactorThroughput = (effProductionVol * 1000) / (hrs * days * numParallelStations)
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
        
        capitalInvestment = (1 - equipDiscount) * baseCapEx * capexFactor
        
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)
        
        materialCosts = h2Gas * h2Price / throughput
        totalMaterialCosts = materialCosts / ((1 - cumRejectionRate) * (1 - h2ScrapRate)) / (1 - (materialScrapRate + polySiRejectRate)) + (otherConsumableCost / (1 - cumRejectionRate))
        
        
        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        return costSummary, reactorThroughput, cumRejectionRate, totalSiPerRod, usableSiPerRod, effProductionVol


def etchFilaments(inUse, siemensCVDRejRate, usableSiPerRod, siemensEffProd):
    """
    Calculates cost of Etch Filaments manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    siemensCVDRejRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    HPProduction : float
        Hair pin production volume
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
        HPProduction = 0
    else:
        
        # define etch filaments input parameters

        HFGas                       = etchFilamentsInputs['Value']['HF Consumption Rate']                               # SCM/hr
        HFPrice                     = etchFilamentsInputs['Value']['HF Price']                                          # $/SCM
        NitricRate                  = etchFilamentsInputs['Value']['Nitric Consumption Rate']                           # kg/hr
        NitricPrice                 = etchFilamentsInputs['Value']['Nitric Acid Price']                                 # $/kg
        N2Gas                       = etchFilamentsInputs['Value']['Nitrogen Consumption Rate']                         # SCM/hr
        N2Price                     = etchFilamentsInputs['Value']['Nitrogen Price']                                    # $/SCM
        DIWater                     = etchFilamentsInputs['Value']['DI Water Consumption Rate']                         # m3
        DIWaterPrice                = etchFilamentsInputs['Value']['DI Water Price']                                    # $/m3
        
        materialScrapRate           = etchFilamentsInputs['Value']['Material Scrap Rate']                               # %
        partRejectRate              = etchFilamentsInputs['Value']['Part Reject Rate']                                  # %
        avgDowntime                 = etchFilamentsInputs['Value']['Average Downtime']                                  # %
        
        filamentBatchSize           = etchFilamentsInputs['Value']['Filament Batch Size']                               # filament/batch
        filamentSetupTime           = etchFilamentsInputs['Value']['Filament Setup Time']                               # min/batch
        filamentCycleTime           = etchFilamentsInputs['Value']['Filament Cycle Time']                               # min/batch
        
        HPBatchSize                 = etchFilamentsInputs['Value']['HP Batch Size']                                     # kg hair pin /batch
        HPSetupTime                 = etchFilamentsInputs['Value']['HP Setup Time']                                     # min/batch
        HPCycleTime                 = etchFilamentsInputs['Value']['HP Cycle Time']                                     # min/batch
        
        baseCapEx                   = etchFilamentsInputs['Value']['Capital Investment']                                # $/station
        
        auxEquipInvest              = TCSInputs['Value']['Auxiliary Equipment Investment']                              # %, Note: In Excel model, set equal to the input value used for TCS mfg step... 
        installFactor               = etchFilamentsInputs['Value']['Installation Cost Factor']
        equipMaintCost              = etchFilamentsInputs['Value']['Equipment Maintenance Cost Factor']                 # %
        
        unskilledDirectLaborers     = etchFilamentsInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor    # ppl/station
        skilledDirectLaborers       = etchFilamentsInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor      # ppl/station
        
        toolinglInvestment          = etchFilamentsInputs['Value']['Tooling Investment']                                # per toolset
        toolingLife                 = etchFilamentsInputs['Value']['Tooling Life']                                      # kgs/tool
        
        operatingPowerConsumption   = etchFilamentsInputs['Value']['Operating Power Consumption']                       # kW
        floorSQM                    = etchFilamentsInputs['Value']['Floor SQM']                                         # sq m
        clean10KSQM                 = etchFilamentsInputs['Value']['Clean 10k SQM']                                     # sq m
        clean1KSQM                  = etchFilamentsInputs['Value']['Clean 1k SQM']                                      # sq m
        clean100SQM                 = etchFilamentsInputs['Value']['Clean 100 SQM']                                     # sq m
        
        virginFilament 				= etchFilamentsInputs['Value']['Fraction Virgin Filament']                          # %
        
        
        # intermediate calculations
        HPProduction = 0    # scrubbed?
        
        cumRejectionRate = 1 - ( (1 - partRejectRate) * (1 - siemensCVDRejRate) )
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        materialCostPerHour = np.dot([HFGas, NitricRate, N2Gas, DIWater], [HFPrice, NitricPrice, N2Price, DIWaterPrice])
        materialCosts = materialCostPerHour * days * hrs / (annualProdVol * 1000)
        totalMaterialCosts = materialCosts / ((1 - cumRejectionRate) * (1 - materialScrapRate))
        
        filamentProcessTime = (filamentCycleTime + filamentSetupTime) / filamentBatchSize / usableSiPerRod
        chunkProcessTime = (HPCycleTime + HPSetupTime) / HPBatchSize
        cycleTime = (filamentProcessTime * virginFilament) + (chunkProcessTime * (1 - virginFilament))
        throughput = 60 / cycleTime
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = (1 - equipDiscount) * baseCapEx * capexFactor * siemensEffProd
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
                
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)


        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        
        return costSummary, cumRejectionRate, HPProduction


def machineFilaments(inUse, etchRejectionRate, totalSiPerRod):
    """
    Calculates cost of Machining Filaments manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    etchRejectionRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
    totalSiPerRod : float
        Total Si mass for each rod (including head/tail)
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
    else:
        
        # define etch filaments input parameters

        consumableRate              = machineFilamentsInputs['Value']['Consumable Rate']                                    # unit/hr
        consumablePrice             = machineFilamentsInputs['Value']['Consumable Price']                                   # $/unit
        
        materialScrapRate           = machineFilamentsInputs['Value']['Material Scrap Rate']                                # %
        yieldLoss                   = machineFilamentsInputs['Value']['Yield Loss']                                         # %
        scrapRecRate                = machineFilamentsInputs['Value']['Scrap Reclamation Rate']
        avgDowntime                 = machineFilamentsInputs['Value']['Average Downtime']                                   # %
        
        filamentBatchSize           = machineFilamentsInputs['Value']['Filament Batch Size']                                # filament/batch
        filamentSetupTime           = machineFilamentsInputs['Value']['Filament Setup Time']                                # min/batch
        filamentCycleTime           = machineFilamentsInputs['Value']['Filament Cycle Time']                                # min/batch
        
        baseCapEx                   = machineFilamentsInputs['Value']['Capital Investment']                                 # $/station
        
        auxEquipInvest              = machineFilamentsInputs['Value']['Auxiliary Equipment Investment']                     # %
        installFactor               = machineFilamentsInputs['Value']['Installation Cost Factor']
        equipMaintCost              = machineFilamentsInputs['Value']['Equipment Maintenance Cost Factor']                  # %
        
        unskilledDirectLaborers     = machineFilamentsInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor     # ppl/station
        skilledDirectLaborers       = machineFilamentsInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor       # ppl/station
        
        toolinglInvestment          = machineFilamentsInputs['Value']['Tooling Investment']                                 # per toolset
        toolingLife                 = machineFilamentsInputs['Value']['Tooling Life']                                       # kgs/tool
        
        operatingPowerConsumption   = machineFilamentsInputs['Value']['Operating Power Consumption']                        # kW
        floorSQM                    = machineFilamentsInputs['Value']['Floor SQM']                                          # sq m
        clean10KSQM                 = machineFilamentsInputs['Value']['Clean 10k SQM']                                      # sq m
        clean1KSQM                  = machineFilamentsInputs['Value']['Clean 1k SQM']                                       # sq m
        clean100SQM                 = machineFilamentsInputs['Value']['Clean 100 SQM']                                      # sq m
        
        
        # intermediate calculations
        
        cumRejectionRate = 1 - (1 - ((1 - scrapRecRate) * yieldLoss)) * (1 - etchRejectionRate)
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        cycleTime = (filamentCycleTime + filamentSetupTime) / filamentBatchSize / totalSiPerRod
        throughput = 60 / cycleTime
        
        materialCosts = consumableRate * consumablePrice * 60 / throughput
        totalMaterialCosts = materialCosts / ((1 - cumRejectionRate) * (1 - materialScrapRate))
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = baseCapEx * capexFactor
        
        ingotWeight =  ingotGrossWeight / (1.15 * growthBatchSize)
        adjEffProdVol = effProductionVol * (ingotWeight / totalSiPerRod)
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, adjEffProdVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
                
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)


        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        
        return costSummary, cumRejectionRate


def sawIngots(inUse, machineRejectRate, usableSiPerRod, totalSiPerRod):
    """
    Calculates cost of Sawing Ingot manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    etchRejectionRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
    usableSiPerRod : float
        Usable mass of Si for each rod
    totalSiPerRod : float
        Total Si mass for each rod (including head/tail)
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
    else:
        
        # define etch filaments input parameters

        wireLife                    = sawIngotsInputs['Value']['Wire Life']                                         # batches
        wireCost                    = sawIngotsInputs['Value']['Wire Cost']                                         # $/wire
        
        SiCSlurryConsumption        = sawIngotsInputs['Value']['SiC Slurry Consumption']                            # kg/hr
        SiCSlurryPrice              = sawIngotsInputs['Value']['SiC Slurry Price']                                  # $/kg
        SiCSlurryScrap              = sawIngotsInputs['Value']['SiC Slurry Scrap Rate']                             # %
        
        kerfLoss                    = sawIngotsInputs['Value']['Kerf Loss']                                         # microns
        sellKerf                    = sawIngotsInputs['Value']['Sell Kerf Loss Si?']                                # 1=Yes, 0=No
        kerfPrice                   = sawIngotsInputs['Value']['Kerf Loss Si Price']                                # $/kg
        partRejectRate              = sawIngotsInputs['Value']['Part Reject Rate']                                  # %
        
        avgDowntime                 = sawIngotsInputs['Value']['Average Downtime']                                  # %
        
        batchSize                   = sawIngotsInputs['Value']['Batch Size']                                        # ingot/batch
        setupTime                   = sawIngotsInputs['Value']['Setup Time']                                        # min/batch
        cutSpeed                    = sawIngotsInputs['Value']['Cutting Speed']                                     # mm/min
        
        baseCapEx                   = sawIngotsInputs['Value']['Capital Investment']                                # $/station
        
        auxEquipInvest              = sawIngotsInputs['Value']['Auxiliary Equipment Investment']                    # %
        installFactor               = sawIngotsInputs['Value']['Installation Cost Factor']
        equipMaintCost              = sawIngotsInputs['Value']['Equipment Maintenance Cost Factor']                 # %
        
        unskilledDirectLaborers     = sawIngotsInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor    # ppl/station
        skilledDirectLaborers       = sawIngotsInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor      # ppl/station
        
        toolinglInvestment          = sawIngotsInputs['Value']['Tooling Investment']                                # per toolset
        toolingLife                 = sawIngotsInputs['Value']['Tooling Life']                                      # kgs/tool
        
        operatingPowerConsumption   = sawIngotsInputs['Value']['Operating Power Consumption']                       # kW
        floorSQM                    = sawIngotsInputs['Value']['Floor SQM']                                         # sq m
        clean10KSQM                 = sawIngotsInputs['Value']['Clean 10k SQM']                                     # sq m
        clean1KSQM                  = sawIngotsInputs['Value']['Clean 1k SQM']                                      # sq m
        clean100SQM                 = sawIngotsInputs['Value']['Clean 100 SQM']                                     # sq m
        
        ingotLength                 = ingotGrowthInputs['Value']['Ingot Length']
        
        
        # intermediate calculations
        
        cumRejectionRate = 1 - ( (1 - partRejectRate) * (1 - machineRejectRate) )
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        cycleTime = ingotLength * 10 / cutSpeed
        processRate = 60 / (cycleTime + setupTime)
        processTime = (cycleTime + setupTime) / (batchSize * growthBatchSize)
        throughput = 60 / (processTime / usableSiPerRod)
        
        sawWireCosts = (wireCost / wireLife) * processRate / throughput
        kerfCosts = -1 * sellKerf * (kerfLoss / 10000)**2 * ingotLength * kerfPrice / (0.9 * totalSiPerRod)
        slurryCost = SiCSlurryConsumption *  SiCSlurryPrice / ( (1 - SiCSlurryScrap) * throughput )
        totalMaterialCosts = np.sum([sawWireCosts, kerfCosts, slurryCost]) / (1 - cumRejectionRate)
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = baseCapEx * capexFactor
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
                
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)


        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        
        return costSummary, cumRejectionRate


def cropIngots(inUse, sawRejectRate, totalSiPerRod):
    """
    Calculates cost of Crop Ingot manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    sawRejectionRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
    totalSiPerRod : float
        Total Si mass for each rod (including head/tail)
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
    else:
        
        # define etch filaments input parameters

        weightRemoved               = cropIngotsInputs['Value']['Weight of Sections Removed']                       # kg
        
        SiCSlurryConsumption        = cropIngotsInputs['Value']['SiC Slurry Consumption']                           # kg/hr
        SiCSlurryPrice              = cropIngotsInputs['Value']['SiC Slurry Price']                                 # $/kg
                
        bladeLife                   = cropIngotsInputs['Value']['Blade Life']                                       # batches
        bladeCost                   = cropIngotsInputs['Value']['Blade Cost']                                       # $/blade
        
        brickRejectRate             = cropIngotsInputs['Value']['Brick Reject Rate']                                # %
        scrapRecRate                = cropIngotsInputs['Value']['Scrap Reclamation Rate']                           # %
        
        avgDowntime                 = cropIngotsInputs['Value']['Average Downtime']                                 # %
        
        batchSize                   = cropIngotsInputs['Value']['Batch Size']                                       # ingot/batch
        setupTime                   = cropIngotsInputs['Value']['Setup Time']                                       # min/batch
        sawRate                     = cropIngotsInputs['Value']['Saw Rate']                                         # mm/min
        
        baseCapEx                   = cropIngotsInputs['Value']['Capital Investment']                               # $/station
        
        auxEquipInvest              = cropIngotsInputs['Value']['Auxiliary Equipment Investment']                   # %
        installFactor               = cropIngotsInputs['Value']['Installation Cost Factor']
        equipMaintCost              = cropIngotsInputs['Value']['Equipment Maintenance Cost Factor']                # %
        
        unskilledDirectLaborers     = cropIngotsInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor   # ppl/station
        skilledDirectLaborers       = cropIngotsInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor     # ppl/station
        
        toolinglInvestment          = cropIngotsInputs['Value']['Tooling Investment']                               # per toolset
        toolingLife                 = cropIngotsInputs['Value']['Tooling Life']                                     # kgs/tool
        
        operatingPowerConsumption   = cropIngotsInputs['Value']['Operating Power Consumption']                      # kW
        floorSQM                    = cropIngotsInputs['Value']['Floor SQM']                                        # sq m
        clean10KSQM                 = cropIngotsInputs['Value']['Clean 10k SQM']                                    # sq m
        clean1KSQM                  = cropIngotsInputs['Value']['Clean 1k SQM']                                     # sq m
        clean100SQM                 = cropIngotsInputs['Value']['Clean 100 SQM']                                    # sq m
        
        ingotDiameter               = ingotGrowthInputs['Value']['As Grown Ingot Diameter']
        
        
        # intermediate calculations
        
        yieldLoss = (weightRemoved / ingotGrossWeight) + brickRejectRate * (1 - scrapRecRate)
        
        cumRejectionRate = 1 - ( (1 - yieldLoss) * (1 - sawRejectRate) )
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        cycleTime = (ingotDiameter / sawRate + setupTime) / batchSize
        throughput = cycleTime * growthBatchSize * totalSiPerRod / 60
        
        wireCosts = bladeCost / (bladeLife * batchSize * ingotGrossWeight)
        slurryCosts = SiCSlurryConsumption * SiCSlurryPrice * (60 / cycleTime) / ingotGrossWeight
        totalMaterialCosts = (wireCosts + slurryCosts) / (1 - cumRejectionRate)
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = baseCapEx * capexFactor
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
                
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)


        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        
        return costSummary, cumRejectionRate


def annealIngots(inUse, cropRejectRate, totalSiPerRod):
    """
    Calculates cost of Anneal Ingot manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    cropRejectRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
    totalSiPerRod : float
        Total Si mass for each rod (including head/tail)
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
    else:
        
        # define input parameters

        argonGas                    = annealIngotsInputs['Value']['Argon Gas']                                          # SLM
        argonPrice                  = annealIngotsInputs['Value']['Argon Price']                                        # $/std liter
        argonScrapRate              = annealIngotsInputs['Value']['Argon Scrap Rate']                                   # %
        
        polySiYieldLossRate         = annealIngotsInputs['Value']['Poly Si Chunk Yield Loss Rate']                      # %
        polySiScrapReclRate         = annealIngotsInputs['Value']['Poly Si Scrap Reclamation Rate']                     # %
        
        avgDowntime                 = annealIngotsInputs['Value']['Average Downtime']                                   # %
        
        furnaceCap                  = annealIngotsInputs['Value']['Furnace Capacity']                                   # kg/hr
        
        baseCapEx                   = annealIngotsInputs['Value']['Capital Investment']                                 # $/station
        
        auxEquipInvest              = annealIngotsInputs['Value']['Auxiliary Equipment Investment']                     # %
        installFactor               = annealIngotsInputs['Value']['Installation Cost Factor']
        equipMaintCost              = annealIngotsInputs['Value']['Equipment Maintenance Cost Factor']                  # %
        
        unskilledDirectLaborers     = annealIngotsInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor     # ppl/station
        skilledDirectLaborers       = annealIngotsInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor       # ppl/station
        
        toolinglInvestment          = annealIngotsInputs['Value']['Tooling Investment']                                 # per toolset
        toolingLife                 = annealIngotsInputs['Value']['Tooling Life']                                       # kgs/tool
        
        operatingPowerConsumption   = annealIngotsInputs['Value']['Operating Power Consumption']                        # kW
        floorSQM                    = annealIngotsInputs['Value']['Floor SQM']                                          # sq m
        clean10KSQM                 = annealIngotsInputs['Value']['Clean 10k SQM']                                      # sq m
        clean1KSQM                  = annealIngotsInputs['Value']['Clean 1k SQM']                                       # sq m
        clean100SQM                 = annealIngotsInputs['Value']['Clean 100 SQM']                                      # sq m
                
        
        # intermediate calculations
        
        cumRejectionRate = 1 - (1 - (1 - polySiScrapReclRate) * polySiYieldLossRate) * (1 - cropRejectRate)
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        filamentWeight = ingotGrossWeight / (1.15 * growthBatchSize)
        throughput = furnaceCap / (filamentWeight / totalSiPerRod)
                
        totalMaterialCosts = (argonGas * argonPrice * 60 / throughput) / ((1 - cumRejectionRate) * (1 - argonScrapRate))
        
        adjEffProdVol = effProductionVol * (filamentWeight / totalSiPerRod)
        runtimeOneStation = runtime(adjEffProdVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = baseCapEx * capexFactor
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
                
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)


        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, operatingPowerConsumption, equipMaintCost)
        
        
        return costSummary, cumRejectionRate


def ingnotGrowth(inUse, annealRejectRate, usableSiPerRod):
    """
    Calculates cost of Ingot Growth manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    annealRejectionRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
    usableSiPerRod : float
        Usable mass of Si for each rod
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    cumRejectionRate : float
        Cumulative rejection rate (waste) for process steps
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
    else:
        
        # define input parameters
        
        polySiPrice                 = ingotGrowthInputs['Value']['Poly-Si Material Price']                          # $/kg
        polySiScrapRate             = ingotGrowthInputs['Value']['Poly Si Material Scrap Rate']                     # %
        
        SiSeedPrice                 = ingotGrowthInputs['Value']['Si Seed Price']                                   # $/pc
        SiSeedLife                  = ingotGrowthInputs['Value']['Si Seed Life']                                    # cycles
        crucibleCost                = ingotGrowthInputs['Value']['Crucible Cost']                                   # $/charge
        crucibleLife                = ingotGrowthInputs['Value']['Crucible Life']                                   # batches/crucible
        waterConsumption            = ingotGrowthInputs['Value']['Cooling Water Cons Rate']                         # m3/batch
        waterPrice                  = ingotGrowthInputs['Value']['Water Price']                                     # $/m3
        argonGas                    = ingotGrowthInputs['Value']['Argon Consumption Rate']                          # SLM
        argonPrice                  = ingotGrowthInputs['Value']['Argon Price']                                     # $/std liter
        partRejectRate              = ingotGrowthInputs['Value']['Part Reject Rate']                                # %
        
        avgDowntime                 = ingotGrowthInputs['Value']['Average Downtime']                                # %
        
        ingotLength                 = ingotGrowthInputs['Value']['Ingot Length']                                    # cm
        setupTime                   = ingotGrowthInputs['Value']['Setup (Load) Time']                               # hrs/batch
        pumpTime                    = ingotGrowthInputs['Value']['Pump Down and Leak Check']                        # hrs/batch
        meltTime                    = ingotGrowthInputs['Value']['Melt/Stabilize']                                  # hrs/batch
        pullSpeed                   = ingotGrowthInputs['Value']['Average Pull Speed']                              # mm/hr
        coolTime                    = ingotGrowthInputs['Value']['Cool and Unload']                                 # hrs/batch
        cleanTime                   = ingotGrowthInputs['Value']['Clean']                                           # hrs/batch
        
        baseCapEx                   = ingotGrowthInputs['Value']['Capital Investment']                              # $/station
        
        auxEquipInvest              = ingotGrowthInputs['Value']['Auxiliary Equipment Investment']                  # %
        installFactor               = ingotGrowthInputs['Value']['Installation Cost Factor']
        equipMaintCost              = ingotGrowthInputs['Value']['Equipment Maintenance Cost Factor']               # %
        
        unskilledDirectLaborers     = ingotGrowthInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor  # ppl/station
        skilledDirectLaborers       = ingotGrowthInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor    # ppl/station
        
        toolinglInvestment          = ingotGrowthInputs['Value']['Tooling Investment']                              # per toolset
        toolingLife                 = ingotGrowthInputs['Value']['Tooling Life']                                    # kgs/tool
        
        operatingPowerConsumption   = ingotGrowthInputs['Value']['Operating Power Consumption']                     # kW
        fullPowerConsumption        = ingotGrowthInputs['Value']['Full Power Consumption']                          # kW
        floorSQM                    = ingotGrowthInputs['Value']['Floor SQM']                                       # sq m
        clean10KSQM                 = ingotGrowthInputs['Value']['Clean 10k SQM']                                   # sq m
        clean1KSQM                  = ingotGrowthInputs['Value']['Clean 1k SQM']                                    # sq m
        clean100SQM                 = ingotGrowthInputs['Value']['Clean 100 SQM']                                   # sq m
                
        
        # intermediate calculations
        
        cumRejectionRate = 1 - (1 - partRejectRate) * (1 - annealRejectRate)
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        filamentWeight = ingotGrossWeight / (1.15 * growthBatchSize)
        
        pullTime = ingotLength * 10 / pullSpeed
        totalCycleTime = np.sum([pullTime, setupTime, pumpTime, meltTime, coolTime, cleanTime])
        productionSpeed = totalCycleTime / growthBatchSize
        
        throughput = usableSiPerRod / productionSpeed
        
        polySiCost = polySiPrice * ingotGrossWeight / (growthBatchSize * (1 - polySiScrapRate))
        SiSeedCost = SiSeedPrice / SiSeedLife / growthBatchSize
        crucibleCost = crucibleCost / crucibleLife / growthBatchSize
        waterCost = waterConsumption * waterPrice / growthBatchSize
        argonCost = argonGas * argonPrice / (productionSpeed * 60)
        totalMaterialCosts = np.sum([polySiCost, SiSeedCost, crucibleCost, waterCost, argonCost]) / (1 - cumRejectionRate) / usableSiPerRod
        
        runtimeOneStation = runtime(effProductionVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = baseCapEx * capexFactor * (1 - equipDiscount)
        
        adjEffProdVol = effProductionVol * filamentWeight / usableSiPerRod
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, adjEffProdVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
                
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)
        
        totalPower = setupTime * operatingPowerConsumption + fullPowerConsumption * (productionSpeed - (setupTime / growthBatchSize))
        adjPowerConsumption = totalPower * throughput / usableSiPerRod

        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, adjPowerConsumption, equipMaintCost)
        
        
        return costSummary, cumRejectionRate


def TCS(inUse, operation, siemensCVDRejRate, HPProduction, siemensEffProd):
    """
    Calculates cost of TCS manfucturing step
    
    Parameters
    ----------
    inUse : bool
        Parameter indicating if this manufacturing step should be included in the process
        True = Included, False = Not inlcuded
    siemensRejectionRate : float
        Cumulative fraction of material wasted in previous manufacturing steps
    HPProduction : float
        Hair pin production amount
    siemensEffProd : float
        Effective production volumne from Siemens CVD step
        
    Returns
    ----------
    costSummary : dataframe object
        Summary of financial cost calculations
    """
    
    if inUse == 'False': 
        costSummary, cumRejectionRate = skipStep()
    else:
        
        # define input parameters
        
        MGSiUsage                   = TCSInputs['Value']['MG Si Usage Factor']                              # kg/kg MG Si
        MGSiPrice                   = TCSInputs['Value']['MG Si Price']                                     # $/kg
        MGSiScrapRate               = TCSInputs['Value']['MG Si Scrap Rate']                                # %
        
        HClUsage                    = TCSInputs['Value']['HCl Usage Factor']                                # kg/kg MG Si
        HClPrice                    = TCSInputs['Value']['HCl Price']                                       # $/pc
        HClScrapRate                = TCSInputs['Value']['HCl Scrap Rate']                                  # %
        
        H2Usage                     = TCSInputs['Value']['Hydrogen Usage Factor']                           # kg/kg MG Si
        H2Price                     = TCSInputs['Value']['Hydrogen Price']                                  # $/kg
        
        otherScrapRate              = TCSInputs['Value']['Other Material Scrap Rate']                       # %
        
        avgDowntime                 = TCSInputs['Value']['Average Downtime']                                # %
                
        auxEquipInvest              = TCSInputs['Value']['Auxiliary Equipment Investment']                  # %
        installFactor               = TCSInputs['Value']['Installation Cost Factor']                        # %
        equipMaintCost              = TCSInputs['Value']['Equipment Maintenance Cost Factor']               # %
        
        unskilledDirectLaborers     = TCSInputs['Value']['Unskilled Direct Laborers Factor'] * laborFactor  # ppl/station
        skilledDirectLaborers       = TCSInputs['Value']['Skilled Direct Laborers Factor'] * laborFactor    # ppl/station
        
        toolinglInvestment          = TCSInputs['Value']['Tooling Investment']                              # per toolset
        toolingLife                 = TCSInputs['Value']['Tooling Life']                                    # kgs/tool
        
        clean10KSQM                 = TCSInputs['Value']['Clean 10k SQM']                                   # sq m
        clean1KSQM                  = TCSInputs['Value']['Clean 1k SQM']                                    # sq m
        clean100SQM                 = TCSInputs['Value']['Clean 100 SQM']                                   # sq m
                
        
        # intermediate calculations
        
        cumRejectionRate = siemensCVDRejRate
        effProductionVol = annualProdVol / (1 - cumRejectionRate)
        
        grossProd = siemensEffProd * 1000
        MGSiCost = ((effProductionVol - HPProduction) / effProductionVol) * grossProd * MGSiPrice * MGSiUsage / (1 - MGSiScrapRate)
        HClCost = grossProd * HClUsage * HClPrice / (1 - HClScrapRate)
        H2Cost = H2Usage * H2Price * grossProd / (1 - otherScrapRate)
        wasteCost = dfSelect(TCSProcessInputs, operation, 'Waste $/kg TCS') * effProductionVol
        totalMaterialCosts = np.sum([MGSiCost, HClCost, H2Cost, wasteCost]) / (1 - cumRejectionRate) / (annualProdVol * 1000)
        
        throughput = (annualProdVol * 1000) / (days * hrs)
        
        adjEffProdVol = effProductionVol * (1 - cumRejectionRate)
        runtimeOneStation = runtime(adjEffProdVol, throughput, avgDowntime)
        numParallelStations = parallelStationCalc(runtimeOneStation)
        
        capitalInvestment = dfSelect(TCSProcessInputs, operation, 'Equipment Cost ($/station)') * capexFactor * (1 - equipDiscount)
        
        prodToolLife = toolLifeCalc(numParallelStations, toolingLife, effProductionVol)
        toolSetPerStation = np.ceil(plantLife / prodToolLife)
        
        operatingPowerConsumption   = dfSelect(TCSProcessInputs, operation, 'Power (kW)') * elecFactor
        heatNatGas                  = dfSelect(TCSProcessInputs, operation, 'Heat Natural Gas (m3/kg Poly Si)') * throughput
        adjUtilities = (operatingPowerConsumption * elecPrice + heatNatGas * natGasPrice) / elecPrice       # adding in Nat Gas and adjusting for use in financial calculation function
        
        unskilledDirectLaborers = unskilledDirectLaborers * annualProdVol / 3000
        skilledDirectLaborers = skilledDirectLaborers * annualProdVol / 3000
        
        floorSQM                    = dfSelect(TCSProcessInputs, operation, 'Floorspace (m2/station)')
        buildingCostperStation = buildingCosts(floorSQM, clean10KSQM, clean1KSQM, clean100SQM)
        
        
        # financial calculations
        
        costSummary = financialCalculations(capitalInvestment, auxEquipInvest, installFactor, runtimeOneStation, toolinglInvestment, toolSetPerStation, buildingCostperStation, numParallelStations, 
                                            totalMaterialCosts, unskilledDirectLaborers, skilledDirectLaborers, throughput, avgDowntime, cumRejectionRate, adjUtilities, equipMaintCost)
        
        
        return costSummary

In [3]:
"""
----- Read Input Data ----- 
"""


scenario                = pd.read_csv('scenario.csv',           index_col=0, header=0).to_dict()

inputs                  = pd.read_csv('indices.csv',            index_col=0, header=0).to_dict()
harvestChunkInputs      = pd.read_csv('harvestChunk.csv',       index_col=0, header=0).to_dict()
siemensCVDInputs        = pd.read_csv('siemensCVD.csv',         index_col=0, header=0).to_dict()
etchFilamentsInputs     = pd.read_csv('etchFilament.csv',       index_col=0, header=0).to_dict()
machineFilamentsInputs  = pd.read_csv('machineFilament.csv',    index_col=0, header=0).to_dict()
sawIngotsInputs         = pd.read_csv('sawIngots.csv',          index_col=0, header=0).to_dict()
cropIngotsInputs        = pd.read_csv('cropIngots.csv',         index_col=0, header=0).to_dict()
annealIngotsInputs      = pd.read_csv('annealIngots.csv',       index_col=0, header=0).to_dict()
ingotGrowthInputs       = pd.read_csv('ingotGrowth.csv',        index_col=0, header=0).to_dict()
TCSInputs               = pd.read_csv('TCS.csv',                index_col=0, header=0).to_dict()
TCSProcessInputs        = pd.read_csv('TCS-process.csv',        index_col=0, header=0)

In [4]:
""" 
----- Read Scenario Input Data ----- 
"""


region              =       scenario['Value']['Region']
annualProdVol       = float(scenario['Value']['Annual Production Volume']       )
plantLife           = float(scenario['Value']['Length of Production Run']       )
dedicatedEquip      =       scenario['Value']['Dedicated Equipment Investment']

harvestChunkInUse   =       scenario['Value']['Use Harvest Chunk?']
siemensCVDInUse     =       scenario['Value']['Use Siemens CVD?']
etchFilamentsInUse  =       scenario['Value']['Use Etch Filaments?']
machineFilInUse     =       scenario['Value']['Use Machine Filaments?']
sawIngotInUse       =       scenario['Value']['Use Saw Ingots?']
cropIngotInUse      =       scenario['Value']['Use Crop Ingots?']
annealIngotInUse    =       scenario['Value']['Use Anneal Ingots?']
growIngotInUse      =       scenario['Value']['Use Grow Ingots?']
TCSInUse            =       scenario['Value']['Use TCS?']

TCSProcess          =       scenario['Value']['TCS Process']

In [5]:
"""
----- Define Regional Constants -----
"""

days                = inputs[region]['Working Days per Year']
hrs                 = inputs[region]['Working Hours per Day']

equipDiscount       = inputs[region]['Equipment Discount']
capexFactor         = inputs[region]['CapEx Correction Factor']
capitalRecRate      = inputs[region]['Capital Recovery Rate']
equipRecLife        = inputs[region]['Equipment Recovery Life']
buildingLife        = inputs[region]['Building Recovery Life']
workingCapPeriod    = inputs[region]['Working Capital Period']

skilledWage         = inputs[region]['Skilled Direct Wages']
unskilledWage       = inputs[region]['Unskilled Direct Wages']
salary              = inputs[region]['Indirect Salary']
indirectLabor       = inputs[region]['Indirect:Direct Labor Ratio']
benefitFactor       = inputs[region]['Benefits on Wage and Salary']
laborFactor         = inputs[region]['Labor Count Multiplier']

natGasPrice         = inputs[region]['Price of Natural Gas']
elecPrice           = inputs[region]['Price of Electricity']
elecFactor          = inputs[region]['Electricity Multiplier']

buildingCostSQF     = inputs[region]['Price of Building Space']
CR10KCost           = inputs[region]['Price of CR10K Building Space']
CR1KCost            = inputs[region]['Price of CR1K Building Space']
CR100Cost           = inputs[region]['Price of CR100Building Space']

In [6]:
""" 
----- Compute Global Constants -----
"""

ingotGrossWeight = grossIngotWeight(ingotGrowthInputs['Value']['Ingot Length'], ingotGrowthInputs['Value']['Ingot Weight (tops and tails)'], ingotGrowthInputs['Value']['As Grown Ingot Diameter'])
growthBatchSize = sizeFilamentBatch(ingotGrowthInputs['Value']['Final Ingot Diameter'], sawIngotsInputs['Value']['Filament Width'], sawIngotsInputs['Value']['Kerf Loss'])

In [7]:
inputObjects = {
    "Harvest Chunk"     : harvestChunkInputs    ,
    "Siemens CVD"       : siemensCVDInputs      ,
    "Etch Filaments"    : etchFilamentsInputs   ,
    "Machine Filaments" : machineFilamentsInputs,
    "Saw Ingots"        : sawIngotsInputs       ,
    "Crop Ingots"       : cropIngotsInputs      ,
    "Anneal Ingots"     : annealIngotsInputs    ,
    "Ingot Growth"      : ingotGrowthInputs     ,
    "TCS"               : TCSInputs             ,
}

In [8]:
def inputToFrame(inputs, label):
    result = reduce(
        lambda x, y: x.join(y),
        [
            pd.DataFrame(dict(((section, k), [v]) for k, v in frame["Value"].items()))
            for section, frame in inputs.items()
        ]
    )
    result.index = pd.Index([label], name="Label")
    return result

In [9]:
def frameToInput(frame, label, target):
    row = frame.loc[label]
    for section, column in row.index:
        if section == "Factor" or section == "Factors":
            continue
        target[section]["Value"][column] = row[(section, column)]

In [10]:
def runModel(inputFrame):

    results = pd.DataFrame()
    
    for label in inputFrame.index:
        
        frameToInput(inputFrame, label, inputObjects)

        harvestFinancials, harvestRejectRate                                                                    = harvestChunk(harvestChunkInUse)
        siemensCVDFinancials, CVDthroughput, siemensCVDRejRate, totalSiPerRod, usableSiPerRod, siemensEffProd   = siemensCVD(siemensCVDInUse, harvestRejectRate)
        etchFilamentsFinancial, etchRejectionRate, HPProduction                                                 = etchFilaments(etchFilamentsInUse, siemensCVDRejRate, usableSiPerRod, siemensEffProd)
        machineFinancials, machineRejectRate                                                                    = machineFilaments(machineFilInUse, etchRejectionRate, totalSiPerRod)
        sawFinancials, sawRejectRate                                                                            = sawIngots(sawIngotInUse, machineRejectRate, usableSiPerRod, totalSiPerRod)
        cropFinancials, cropRejectRate                                                                          = cropIngots(cropIngotInUse, sawRejectRate, totalSiPerRod)
        annealFinacials, annealRejectRate                                                                       = annealIngots(annealIngotInUse, cropRejectRate, totalSiPerRod)
        growthFinancials, growthRejRate                                                                         = ingnotGrowth(growIngotInUse, annealRejectRate, usableSiPerRod)
        TCSFinancials = TCS(TCSInUse, TCSProcess, siemensCVDRejRate, HPProduction, siemensEffProd)
    
        steps = [harvestFinancials, siemensCVDFinancials, etchFilamentsFinancial, machineFinancials, sawFinancials, cropFinancials, annealFinacials, growthFinancials, TCSFinancials]
        result = financialSummary(steps)
        
        result = result[["$/kg Poly Si Chunk"]].transpose()
        result.index = [label]
        result.index.name = "Label"
        results = results.append(result)

    results.columns = pd.MultiIndex.from_tuples([("Results", column) for column in results.columns])
    return results

In [11]:
defaults = inputToFrame(inputObjects, 0)

In [12]:
experiment = deepcopy(defaults)
experiment.insert(0, ("Factor", ""), "default", True)

n = 0

for section, column in defaults.columns:
    for m in [exp(x) for x in np.arange(log(0.1), log(10), log(101) / 101)]:
        n += 1
        row = deepcopy(defaults)
        row.insert(0, ("Factor", ""), "default", True)
        row.index = [n]
        value = row.at[n, (section, column)]
        if value == 0:
            break
        row.at[n, (section, column)] = m * value
        row.at[n, ("Factor", "")] = section + " | " + column
        experiment = experiment.append(row)

%time results = runModel(experiment)

/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: `item` has been deprecated and will be removed in a future version
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:804: RuntimeWarning: divide by zero encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6

CPU times: user 17min 12s, sys: 242 ms, total: 17min 12s
Wall time: 17min 12s


In [13]:
results.shape

(18585, 10)

In [14]:
experiment.join(results).to_csv("experiment-results.tsv", sep="\t")

In [56]:
sensitiveFactors = pd.read_csv("sensitive-factors.tsv", sep="\t")
sensitiveFactors

,Section,Column
0,Etch Filaments,Fraction Virgin Filament
1,Ingot Growth,Poly Si Material Scrap Rate
2,Siemens CVD,Final Rod Size
3,TCS,MG Si Price
4,TCS,MG Si Usage Factor
5,Ingot Growth,Ingot Length
6,Siemens CVD,Cycle Time
7,Siemens CVD,Batch Size
8,Siemens CVD,Hydrogen Consumption Rate
9,Siemens CVD,Hydrogen Price


In [16]:
experiment2 = deepcopy(defaults)
experiment2.insert(0, ("Factors", ""), "default", True)

n = 1

for section1, column1 in defaults.columns:
    if not any((section1 == sensitiveFactors.Section) & (column1 == sensitiveFactors.Column)):
        continue
    for section2, column2 in defaults.columns:
        if section1 == section2 and column1 >= column2:
            continue
        if not any((section2 == sensitiveFactors.Section) & (column2 == sensitiveFactors.Column)):
            continue
        for m1 in [exp(x) for x in np.arange(log(0.1), log(10), log(100) / 31)]:
            for m2 in [exp(x) for x in np.arange(log(0.1), log(10), log(100) / 31)]:
                row = deepcopy(defaults)
                row.insert(0, ("Factors", ""), "default", True)
                row.index = [n]
                value1 = row.at[n, (section1, column1)]
                if value1 == 0:
                    break
                row.at[n, (section1, column1)] = m1 * value1
                value2 = row.at[n, (section2, column2)]
                if value2 == 0:
                    break
                row.at[n, (section2, column2)] = m2 * value2
                row.at[n, ("Factors", "")] = section1 + " | " + column1 + " & " + section2 + " | " + column2
                experiment2 = experiment2.append(row)
                n += 1

%time results2 = runModel(experiment2)

/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: `item` has been deprecated and will be removed in a future version
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:418: RuntimeWarning: divide by zero encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6-ipykernel-5.1.1/lib/python3.6/site-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in double_scalars
/nix/store/aq550jlky0l0m5vg66inb0gfyiin045f-python3.6

CPU times: user 11h 55min 50s, sys: 14min 7s, total: 12h 9min 58s
Wall time: 12h 9min 43s


In [17]:
results2.shape

(725556, 10)

In [18]:
experiment2.join(results2).to_csv("experiment-results2.tsv", sep="\t")

In [251]:
results1 = pd.read_csv("experiment-results.tsv", sep="\t", header=[0,1], index_col=0)
results1.columns = ["Factor"] + [x[0] + " @ " + x[1] for x in results1.columns.values if x[0] != "Factor"]
results1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3865 entries, 0 to 3907
Columns: 241 entries, Factor to Results @ Total
dtypes: float64(240), object(1)
memory usage: 7.1+ MB


In [252]:
defaults = results1[results1.Factor == "default"].iloc[0]
defaults

Factor                                 default
Harvest Chunk @ Argon Usage                 10
Harvest Chunk @ Argon Price                0.1
Harvest Chunk @ Argon Scrap Rate          0.05
Harvest Chunk @ Poly Si Yield Loss        0.01
                                        ...   
Results @ Building Cost               0.155338
Results @ Maintenance Cost             1.09004
Results @ Overhead Labor Cost         0.623845
Results @ Cost of Capital              3.73962
Results @ Total                        31.2733
Name: 0, Length: 241, dtype: object

In [268]:
def f1(row):
    iname = row["Factor"]
    ivalue = row[iname]
    ideviation = log(ivalue / defaults[iname])
    result = max(row["Results @ Total"], 0.01)
    deviation = log(result / defaults["Results @ Total"])
    return pd.Series([
        iname,
        ivalue,
        ideviation,
        round(abs(ideviation * 10 / log(10))),
        result,
        deviation
    ])

In [269]:
z1 = results1[results1["Factor"] != "default"].apply(f1, axis=1)
z1.columns=[
    "Factor",
    "Value",
    "Log Change",
    "Index",
    "Result",
    "Log Change Result",
]

In [272]:
def f0(row):
    iname = row["Factor"]
    ivalue = row[iname]
    ideviation = log(ivalue / defaults[iname])
    result = row["Results @ Total"]
    return pd.Series([
        iname,
        ivalue,
        ideviation,
        np.sign(ideviation),
        result,
        result <= defaults["Results @ Total"],
    ])

In [273]:
z0 = results1[results1["Factor"] != "default"].apply(f0, axis=1)
z0.columns=[
    "Factor",
    "Value",
    "Log Change",
    "Sign",
    "Result",
    "Improvement?",
]

In [284]:
z0.shape

(3864, 6)

In [285]:
z1.shape

(3864, 6)

In [286]:
signs.shape

(184, 2)

In [291]:
signs.merge(z1, on=["Factor"], how="inner").shape

(3864, 7)

In [274]:
signs = z0.loc[z0.groupby("Factor").Result.idxmin()].sort_index()[["Factor", "Sign"]]

In [292]:
smerge = signs.merge(z1, on=["Factor"], how="inner")
smerge = smerge[(smerge.Sign == np.sign(smerge["Log Change"])) | (smerge.Index == 0)]
smerge.shape

(2024, 7)

In [364]:
results2 = pd.read_csv("experiment-results2.tsv", sep="\t", header=[0,1], index_col=0)
results2.columns = ["Factors"] + [x[0] + " @ " + x[1] for x in results2.columns.values if x[0] != "Factors"]
results2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332955 entries, 1 to 332955
Columns: 241 entries, Factors to Results @ Total
dtypes: float64(240), object(1)
memory usage: 614.7+ MB


In [365]:
def f2(row):
    iname, jname = row[0].split(" & ")
    ivalue = row[iname]
    jvalue = row[jname]
    ideviation = log(ivalue / defaults[iname])
    jdeviation = log(jvalue / defaults[jname])
    result = max(row["Results @ Total"], 0.01)
    deviation = log(result / defaults["Results @ Total"])
    return pd.Series([
        iname,
        jname,
        ivalue,
        jvalue,
        ideviation,
        jdeviation,
        round(abs(ideviation * 10 / log(10))),
        round(abs(jdeviation * 10 / log(10))),
        result,
        deviation
    ])

In [366]:
z2 = results2.apply(f2, axis=1)
z2.columns=[
    "Factor 1",
    "Factor 2",
    "Value 1",
    "Value 2",
    "Log Change 1",
    "Log Change 2",
    "Index 1",
    "Index 2",
    "Result",
    "Log Change Result",
]
z2 = z2[z2["Factor 1"] < z2["Factor 2"]]

In [367]:
z2.shape[0] / 21 / 21 / (30 * 29 / 2)

1.0

In [389]:
z3 = signs.merge(z2, left_on="Factor", right_on="Factor 1", how="inner")
z3 = z3[(z3.Sign == np.sign(z3["Log Change 1"])) | (z3["Index 1"] == 0)][["Factor 1", "Factor 2", "Value 2", "Log Change 2", "Index 1", "Index 2", "Result", "Log Change Result"]]
z3 = signs.merge(z3, left_on="Factor", right_on="Factor 2", how="inner")
z3 = z3[(z3.Sign == np.sign(z3["Log Change 2"])) | (z3["Index 2"] == 0)][["Factor 1", "Factor 2", "Index 1", "Index 2", "Result", "Log Change Result"]]
z3.shape

(52635, 6)

In [390]:
z3 = pd.concat([z3, z3.rename(columns={
    "Factor 1" : "Factor 2",
    "Factor 2" : "Factor 1",
    "Index 1" : "Index 2",
    "Index 2" : "Index 1",
})[["Factor 1", "Factor 2", "Index 1", "Index 2", "Result", "Log Change Result"]]])
z3 = z3.set_index(["Factor 1", "Index 1", "Factor 2", "Index 2"])

In [391]:
z3.shape[0] / 11 / 11 / 30 / 29

1.0

In [440]:
def smpl1():
        return pd.DataFrame([
            pd.Series({
                "Factor 1" : factor,
                "Index 1" : np.random.choice(list(range(0,11)), p=[0.290, 0.025, 0.050, 0.075, 0.100, 0.105, 0.105, 0.100, 0.075, 0.050, 0.025])
            })
            for factor in z2["Factor 1"].unique()
        ]).set_index(["Factor 1", "Index 1"])
def smpl2():
        return pd.DataFrame([
            pd.Series({
                "Factor 2" : factor,
                "Index 2" : np.random.choice(list(range(0,11)), p=[0.290, 0.025, 0.050, 0.075, 0.100, 0.105, 0.105, 0.100, 0.075, 0.050, 0.025])
            })
            for factor in z2["Factor 1"].unique()
        ]).set_index(["Factor 2", "Index 2"])    

In [441]:
z3.head()

Result  \
Factor 1                                 Index 1 Factor 2                                       Index 2              
Harvest Chunk @ Installation Cost Factor 10      Harvest Chunk @ Poly Si Scrap Reclemation Rate 0        30.481680   
                                                                                                1        30.403529   
                                                                                                2        30.305867   
                                                                                                3        30.184049   
                                                                                                4        30.032457   

                                                                                                         Log Change Result  
Factor 1                                 Index 1 Factor 2                                       Index 2                     
Harvest Chunk @ Installation Cost Factor 10      Harvest Chunk @ Poly Si Scrap Reclemation Rate 0                -0.025640  
                                                                                                1                -0.028208  
                                                                                                2                -0.031425  
                                                                                                3                -0.035453  
                                                                                                4                -0.040487

In [442]:
smpl1().head()

,
Factor 1,Index 1
Harvest Chunk @ Poly Si Scrap Reclemation Rate,8
Harvest Chunk @ Installation Cost Factor,0
Harvest Chunk @ Unskilled Direct Laborers Factor,3
Harvest Chunk @ Skilled Direct Laborers Factor,7
Harvest Chunk @ Operating Power Consumption,1


In [548]:
okay = z3[(z3["Result"] > 5) & (z3["Result"] < 31)]
def smpl():
    z4 = okay.join(smpl2(), how="inner").join(smpl1(), how="inner")
    f1 = z4.Result.idxmin()[0]
    z5 = okay.loc[[f1]]
    z6 = z5.loc[z5.groupby(["Factor 1", "Index 1"]).Result.idxmin()].droplevel("Index 2").drop(["Result", "Log Change Result"], axis=1)
    return okay.join(z6, how="inner").join(smpl2(), how="inner")

In [549]:
smpl()

Result  \
Factor 2                                  Index 2 Factor 1          Index 1              
Etch Filaments @ Fraction Virgin Filament 4       TCS @ MG Si Price 1        18.667939   
                                                                    2        17.026936   
                                                                    3        15.723441   
                                                                    5        13.865588   
                                                                    6        13.212293   
                                                                    7        12.693362   
                                                                    8        12.281160   
                                                                    9        11.953737   
                                                                    10       11.693656   
TCS @ MG Si Scrap Rate                    0       TCS @ MG Si Price 4        25.227545   

                                                                             Log Change Result  
Factor 2                                  Index 2 Factor 1          Index 1                     
Etch Filaments @ Fraction Virgin Filament 4       TCS @ MG Si Price 1                -0.515959  
                                                                    2                -0.607970  
                                                                    3                -0.687614  
                                                                    5                -0.813356  
                                                                    6                -0.861619  
                                                                    7                -0.901687  
                                                                    8                -0.934700  
                                                                    9                -0.961722  
                                                                    10               -0.983720  
TCS @ MG Si Scrap Rate                    0       TCS @ MG Si Price 4                -0.214830

In [559]:
%time result3 = pd.concat([smpl() for i in range(0, 5000)]).reset_index()[["Factor 1", "Factor 2", "Index 1", "Index 2", "Result"]]

CPU times: user 41min 34s, sys: 0 ns, total: 41min 34s
Wall time: 41min 33s


In [560]:
result3["Factor 1"].unique()

array(['TCS @ MG Si Scrap Rate',
       'Etch Filaments @ Fraction Virgin Filament',
       'Siemens CVD @ Hydrogen Scrap Rate', 'TCS @ MG Si Usage Factor',
       'Ingot Growth @ Ingot Length', 'TCS @ MG Si Price',
       'TCS @ Installation Cost Factor', 'Siemens CVD @ Final Rod Size',
       'Siemens CVD @ Hydrogen Consumption Rate',
       'Harvest Chunk @ Capital Investment', 'Siemens CVD @ Batch Size',
       'Siemens CVD @ Capital Investment',
       'Siemens CVD @ Installation Cost Factor',
       'Siemens CVD @ Cycle Time', 'Siemens CVD @ Hydrogen Price',
       'Saw Ingots @ Kerf Loss', 'Siemens CVD @ Other Consumable Cost'],
      dtype=object)

In [561]:
result3.to_csv("results3.tsv", sep="\t")

In [107]:
improvements = results2.loc[results2['Results | Total'] <= defaults['Results | Total']]
improvements = improvements[1:]
improvements.shape

(345392, 241)

In [62]:
investments = pd.DataFrame({
    "Factor"    : [row.Section + " | " + row.Column for _, row in sensitiveFactors.iterrows()],
    "Low Cost"  : [random.uniform(a=0.75, b=1.00)   for _, row in sensitiveFactors.iterrows()],
    "High Cost" : [random.uniform(a=1.00, b=1.25)   for _, row in sensitiveFactors.iterrows()],
})
investments.sort_values("Factor")

,Factor,Low Cost,High Cost
0,Etch Filaments | Fraction Virgin Filament,0.787502,1.063794
17,Harvest Chunk | Capital Investment,0.827283,1.190584
20,Harvest Chunk | Installation Cost Factor,0.877592,1.189904
23,Harvest Chunk | Operating Power Consumption,0.894390,1.169793
14,Harvest Chunk | Poly Si Scrap Reclemation Rate,0.844111,1.121535
24,Harvest Chunk | Skilled Direct Laborers Factor,0.918464,1.000195
19,Harvest Chunk | Unskilled Direct Laborers Factor,0.878549,1.155058
5,Ingot Growth | Ingot Length,0.925697,1.158831
1,Ingot Growth | Poly Si Material Scrap Rate,0.928167,1.163846
21,Saw Ingots | Kerf Loss,0.876894,1.047669


In [93]:
def computeCost(x):
    return sum(
        [
            random.uniform(a=row["Low Cost"], b=row["High Cost"]) * abs(log(x[row.Factor] / defaults[row.Factor]))
            for _, row in investments.iterrows()
        ]
    )

In [113]:
def computeBenefit(x):
    return 1 - x['Results | Total'] / defaults['Results | Total']

In [124]:
best = pd.DataFrame({
    "Factors"      : improvements.Factors,
    "Benefit/Cost" : [computeBenefit(x) / computeCost(row) for _, row in improvements.iterrows()]
}, index=improvements.index).sort_values("Benefit/Cost", ascending=False)

In [125]:
best.to_csv("best.tsv", sep="\t")